In [5]:
llm_model = "accounts/fireworks/models/llama-v3p1-70b-instruct"
embedding_model = "nomic-ai/nomic-embed-text-v1.5"

In [73]:
import pandas as pd

df = pd.read_csv("OutdoorClothingCatalog_1000.csv", index_col=[0])
df

,name,description
0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
1,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
2,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
3,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
4,EcoFlex 3L Storm Pants,Our new TEK O2 technology makes our four-seaso...
...,...,...
995,"Men's Classic Denim, Standard Fit",Crafted from premium denim that will last wash...
996,CozyPrint Sweater Fleece Pullover,The ultimate sweater fleece - made from superi...
997,Women's NRS Endurance Spray Paddling Pants,These comfortable and affordable splash paddli...
998,Women's Stop Flies Hoodie,This great-looking hoodie uses No Fly Zone Tec...


In [79]:
product_review_list = df.apply(
    lambda row: f"Product name: {row['name']} - Description: {row['description']}",
    axis=1,
).tolist()
product_review_list = product_review_list[:100]

In [80]:
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_fireworks import ChatFireworks
from langchain_fireworks import FireworksEmbeddings

llm = ChatFireworks(model=llm_model, temperature=0.5)
embed_model = FireworksEmbeddings(model=embedding_model)

vectorstore = Chroma.from_texts(product_review_list, embedding=embed_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [89]:
query = "Please suggest a shirt with sunblocking"

In [90]:
print(rag_chain.invoke(query))

Based on the provided context, I would suggest the following shirts with sunblocking features:

1. Women's Tropical Plaid Shirt - This shirt has a UPF 50+ rating, which is the highest rated sun protection possible.
2. Performance Plus Woven Shirt - This shirt has a UPF 40+ rating, which blocks the sun's UV rays.
3. Rangeley Camp Performance Checked Flannel Shirt - Although not specifically mentioned as a sunblocking feature, the high-performance hollow-core polyester yarns in this shirt are designed to wick away moisture and dry fast, which may provide some level of sun protection.

However, if you're looking for a shirt with the highest rated sun protection, I would recommend the Women's Tropical Plaid Shirt.


In [85]:
query = " I need a throw pillow insert that fluffs up well. Any recommendations?"

In [87]:
print(rag_chain.invoke(query))

Based on the reviews, I would highly recommend the "Pillows Insert" product. Every single review (all 7 of them) says that it's the best throw pillow filler on Amazon, and that it fluffs up great after being tossed in the dryer. They also mention that it's a big improvement over other cheap and flat inserts they've tried.


In [93]:
query = (
    "Please list all your shirts with sun protection in a table and summarize each one."
)

In [94]:
print(rag_chain.invoke(query))

Here is the list of shirts with sun protection in a table format:

| **Product Name** | **Description** | **Sun Protection** |
| --- | --- | --- |
| Women's Tropical Plaid Shirt | Lightweight, breathable shirt for hot weather | UPF 50+ rated |
| Performance Plus Woven Shirt | Breathable summer shirt with quick-dry fabric | UPF 40+ rated (but provides SPF 50+ sun protection) |
| Smooth Comfort Check Shirt | Slightly fitted check shirt with wrinkle-free performance | No explicit sun protection rating, but has TrueCool fabric that wicks moisture away from skin |

Note that the Rangeley Camp Performance Checked Flannel Shirt has a UPF rating, but it's not explicitly stated as providing sun protection, so I didn't include it in the table. The Easy-Care Classic Stripe Short-Sleeve Shirt and the Soft Cotton Colorblock Crew-Neck Top do not have any explicit sun protection features.

Here's a brief summary of each shirt:

* Women's Tropical Plaid Shirt: A lightweight, breathable shirt designed 